# Logistic Regression

Using GPU-accelerated logistic regression to predict infection risk based on features of the population members.

## Imports

In [1]:
import cudf
import cuml

import cupy as cp

## Load Data

In [2]:
gdf = cudf.read_csv('./data/pop_2-05.csv', usecols=['age', 'sex', 'infected'])

In [3]:
gdf.dtypes

age         float64
sex         float64
infected    float64
dtype: object

In [4]:
gdf.shape

(58479894, 3)

In [5]:
gdf.head()

,age,sex,infected
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


## Logistic Regression

We would like to estimate infection risk based on population members' age and sex using logistic Regression

In [7]:
#Here we create a cuML logistic regression instance `logreg`:
logreg = cuml.LogisticRegression()

## Exercise: Regress Infected Status

The `logreg.fit` method takes 2 arguments: the model's independent variables *X*, and the dependent variable *y*. Fit the `logreg` model using the `gdf` columns `age` and `sex` as *X* and the `infected` column as *y*.

In [9]:
logreg.fit(gdf[['age', 'sex']], gdf['infected'])

LogisticRegression(penalty='l2', tol=0.0001, C=1.0, fit_intercept=True, max_iter=1000, linesearch_max_iter=50, verbose=4, l1_ratio=None, solver='qn', handle=<cuml.common.handle.Handle object at 0x7f19eaee3af0>, output_type='cudf')

## Viewing the Regression

After fitting the model, we could use `logreg.predict` to estimate whether someone has more than a 50% chance to be infected, but since the virus has low prevalence in the population (around 1-2%, in this dataset), individual probabilities of infection are well below 50% and the model should correctly predict that no one is individually likely to have the infection.

Here we view these values. Notice that changing sex from 0 to 1 has the same effect via the coefficients as changing the age by ~48 years.

In [12]:
logreg_coef = logreg.coef_
logreg_int = logreg.intercept_

print("Coefficients: [age, sex]")
print([logreg_coef[0], logreg_coef[1]])

print("Intercept:")
print(logreg_int[0])

Coefficients: [age, sex]
[0.01470146729444174, 0.7002792201933117]
Intercept:
-5.215624611661989


## Estimate Probability of Infection

Calculating the estimated percentage risk of infection.

In [13]:
class_probs = logreg.predict_proba(gdf[['age', 'sex']])
class_probs

,0,1
0,0.994598,0.005402
1,0.994598,0.005402
2,0.994598,0.005402
3,0.994598,0.005402
4,0.994598,0.005402
...,...,...
58479889,0.960540,0.039460
58479890,0.960540,0.039460
58479891,0.960540,0.039460
58479892,0.960540,0.039460


Remembering that a 1 indicates 'infected', we assign that class' probability to a new column in the original dataframe:

In [14]:
gdf['risk'] = class_probs[1]

Looking at the original records with their new estimated risks, we can see how estimated risk varies across individuals.

In [15]:
gdf.take(cp.random.choice(gdf.shape[0], size=5, replace=False))

,age,sex,infected,risk
26593127,74.0,0.0,0.0,0.015864
7419339,20.0,0.0,0.0,0.007235
45391013,45.0,1.0,0.0,0.020759
6764162,19.0,0.0,0.0,0.007130
207416,0.0,0.0,0.0,0.005402


## Exercise: Show Infection Prevalence is Related to Age

The positive coefficient on age suggests that the virus is more prevalent in older people, even when controlling for sex.

Showing that infection prevalence has some relationship to age by printing the mean `infected` values for the oldest and youngest members of the population when grouped by age:

In [17]:
age_groups = gdf[['age', 'infected']].groupby(['age'])
print(age_groups.mean().head())
print(age_groups.mean().tail())

     infected
age          
0.0  0.000000
1.0  0.000889
2.0  0.001960
3.0  0.002715
4.0  0.003586
      infected
age           
86.0  0.023417
87.0  0.023256
88.0  0.024569
89.0  0.024412
90.0  0.025017


## Exercise: Show Infection Prevalence is Related to Sex

Similarly, the positive coefficient on sex suggests that the virus is more prevalent in people with sex = 1 (females), even when controlling for age.

Showing that infection prevalence has some relationship to sex by printing the mean `infected` values for the population when grouped by sex:

In [19]:
sex_groups = gdf[['sex', 'infected']].groupby(['sex'])
print(sex_groups.mean())

     infected
sex          
0.0  0.010140
1.0  0.020713


## Making Predictions with Separate Training and Test Data

cuML gives us a simple method for producing paired training/testing data:

In [22]:
X_train, X_test, y_train, y_test  = cuml.train_test_split(gdf[['age', 'sex']], gdf['infected'], train_size=0.9)

## Exercise: Fit Logistic Regression Model Using Training Data

In [23]:
logreg = cuml.LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(penalty='l2', tol=0.0001, C=1.0, fit_intercept=True, max_iter=1000, linesearch_max_iter=50, verbose=4, l1_ratio=None, solver='qn', handle=<cuml.common.handle.Handle object at 0x7f19e8d3cd70>, output_type='cudf')

## Use Test Data to Validate Model

We can now use the same procedure as above to predict infection risk using the test data:

In [24]:
y_test_pred = logreg.predict_proba(X_test, convert_dtype=True)[1]
y_test_pred.index = X_test.index
y_test_pred

50301085    0.024928
23016541    0.012854
2167022     0.006119
55567082    0.031220
24799804    0.013933
              ...   
12851910    0.008934
30310809    0.012269
34459256    0.014416
30515467    0.012269
58429194    0.038529
Name: 1, Length: 5847990, dtype: float64

As we saw before, very few people are actually infected in the population, even among the highest-risk groups. As a simple way to check our model, we split the test set into above-average predicted risk and below-average predicted risk, then observe that the prevalence of infections correlates closely to those predicted risks.

In [25]:
test_results = cudf.DataFrame()
test_results['age'] = X_test['age']
test_results['sex'] = X_test['sex']
test_results['infected'] = y_test
test_results['predicted_risk'] = y_test_pred

test_results['high_risk'] = test_results['predicted_risk'] > test_results['predicted_risk'].mean()

risk_groups = test_results.groupby('high_risk')
risk_groups.mean()

,age,sex,infected,predicted_risk
high_risk,,,,
False,30.592640,0.223311,0.009844,0.010462
True,54.147839,0.921533,0.023892,0.023384


Finally, in a few milliseconds, we can do a two-tier analysis by sex and age:

In [ ]:
%%time
s_groups = test_results[['sex', 'age', 'infected', 'predicted_risk']].groupby(['sex', 'age'])
s_groups.mean()